# 空间噪音生成函数

图像的复原与重建也是图像学领域的重要技术，除了可以用于老照片的恢复外，也可以用于图像信号的增强与去噪。在前面的章节中，虽然已经介绍过三种比较常见的噪音，与其生成方法。但是在我们进行这方面的研究中，有时候也会使用一些其他噪音生成方法。

在老爷子的教科书里提到了大概以下6种不同的噪音信号[^1]：

尽管从我个人的经验看还有一些不是很常见的噪音，不过这6种主要的噪音类型已经足以涵盖我们认知的绝大部分类型了。

如果对于噪音怎么生成不是很理解的朋友，可以看看我前面的文章

## 高斯噪音函数

$$
p(z) = \frac{1}{ \sqrt {2 \pi } \sigma } e^(- \frac{(z- \mu )^2}{2 \sigma^2} )
$$

其中$\mu$表示数学期望，$\sigma$表示标准差。

## 瑞利噪音函数

高斯、瑞利、伽马、指数噪音基本都是e指数分布家族，但是在具体形式上还是有很大的不同，这里直接从公式摘抄了：

$$
p(z) = \left\{\begin{matrix}
\frac{2}{b}(z-a)e^{-(z-a)^2/b} &, z \geqslant  a \\ 
0,& z < a
\end{matrix}\right.$$

而他的概率均值由

$$
\bar{z} = a + \sqrt{\pi b / 4}
$$

以及方差：

$$
\sigma^2 = \frac{b (4 - \pi)}{4}
$$

## 伽马噪音函数

$$
p(z)=\left\{\begin{matrix} 
\frac{a^b z^{b-1}}{(b-1)!} e^{-az}, & z \geqslant  a \\
0, & z < a
\end{matrix}\right.
$$

其中，参数a > 0, b为正整数，其均值和方差分别如下：

$$
\bar{z} = \frac{1}{a}
$$

和
$$
\sigma^2 = \frac{1}{a^2}
$$

## 均匀噪音
$$
p(z) = \left\{\begin{matrix}
\frac{1}{b -a} & a \leqslant z \leqslant b \\
0, & elsewise
\end{matrix}\right.
$$

他的密度函数均值和方差分别如下：
 
$$
\bar{z} = \frac{a+b}{2}
$$

$$
\sigma^2 = \frac{(b-a)^2}{12}
$$

## 椒盐噪音

$$
p(z) = \left\{\begin{matrix} 
P_a, & z=a \\
1-P_a, & elsewise
\end{matrix}\right.
$$

提一个问题，有必要记住这些噪音函数吗？其实是不需要的，你需要知道的是这些噪音函数的特点，以及行业内通常的处理经验，以便在自己实际工作中遇到后，能想办法去除噪音的影响。所以我们要理解噪音对图像造成的退化影响，就需要做一些小实验


# 噪音对图像退化影响

我比较好奇，原书中作者给出的测试图在未经过噪音函数影响前的实际直方图是什么样子：

![在这里插入图片描述](https://img-blog.csdnimg.cn/20210417164741362.jpg#pic_center)

我们用前面章节里实现过的直方图计算函数来plot一下：

In [ ]:
import numpy as np


def calculate_hist(img):
    histogram = np.zeros(256, dtype=np.uint64)
    
    row, col = img.shape
    for r in range(row):
        for c in range(col):
            histogram[img[r, c]] = histogram[img[r, c]] + 1

    return histogram


当然，你也可以直接使用matplotlib的hist函数绘制直方图，只不过以我个人的实际运行状况来看，那个运行结果会特别缓慢。

![在这里插入图片描述](https://img-blog.csdnimg.cn/20210419143045622.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3BvaXNvbmNocnk=,size_16,color_FFFFFF,t_70#pic_center)
![在这里插入图片描述](https://img-blog.csdnimg.cn/20210419143057449.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3BvaXNvbmNocnk=,size_16,color_FFFFFF,t_70#pic_center)

# 观察结果
如果一副图像只是包含一些空间噪音，那相对来说是比较好发现的。因为从数学上来说，噪音只是对原始图像的线型相加，用公式表达就是

$$g(x, y) = f(x, y) + \eta(x, y)$$

要剔除噪音对我们的影响，我们可以先输出图像的直方图，然后分析噪音信号的特点后，从以上常见的6个噪音函数中挑选出可能的一两个，通过调整参数的方法建模。然后执行以下逆运算基本就可以得到一张剔除了噪音信号的图像。

$$f(x, y) = g(x, y) - \eta(x, y)$$

其中，$\eta$ 表示噪音函数。但是一张泛黄模糊的老照片可不止单单剔除噪音这么简单，比如在拍照时的镜头是否聚焦，拍照时物体或者镜头是否发生了晃动，如果用数码设备进行采集图像，那么CMOS的温度、湿度，当时当地是否出现了强磁场干扰等原因，都会导致最终拍摄的照片出现模糊不清的问题，不过在正式介绍基于以上原因而采用的图像复原方法前，先会再介绍一些由于频域干扰而对图像质量产生的影响。

[^1]: 《数字图像处理》，冈萨雷斯